In [198]:
from traitlets import Unicode, validate, Bytes
from traittypes import Array
import numpy as np

import ipywidgets as widgets
from IPython.display import display


def array_to_json(ar, obj=None):
	return ar.tolist() if ar is not None else None

def from_json_to_array(value, obj=None):
	return np.array(value) if value else None


array_serialization = dict(to_json=array_to_json, from_json=from_json_to_array)

class HelloWidget(widgets.DOMWidget):
    _view_name = Unicode('HelloView').tag(sync=True)
    _view_module = Unicode('hello').tag(sync=True)
    array_bigv = Array(default_value=None,allow_none=True).tag(sync=True,**array_serialization)
    byte_bigv = Bytes().tag(sync=True)
    

In [262]:
%%javascript
require.undef('hello');

function asciiDecode(buf) {
        return String.fromCharCode.apply(null, new Uint8Array(buf));
}

function readUint16LE(buffer) {
        var view = new DataView(buffer);
        var val = view.getUint8(0);
        val |= view.getUint8(1) << 8;
        return val;
}

function fromArrayBuffer(buf) {
      // Check the magic number
      //var buf = new ArrayBuffer(buf)
      //console.log(buf)
    
      console.log("l",buf.slice(1,6) )

      var magic = asciiDecode(buf.slice(0,6));
      console.log("La")
      //return "test"
      console.log("l",magic.slice(1,6) )
      if (magic.slice(1,6) != 'NUMPY') {
          throw new Error('unknown file type');
      }

      var version = new Uint8Array(buf.slice(6,8)),
          headerLength = readUint16LE(buf.slice(8,10)),
          headerStr = asciiDecode(buf.slice(10, 10+headerLength));
          var offsetBytes = 10 + headerLength;
          //rest = buf.slice(10+headerLength);  XXX -- This makes a copy!!! https://www.khronos.org/registry/typedarray/specs/latest/#5

      console.log(headerStr.toLowerCase().replace('(','[').replace(',),',']').replace('),',']').replace(/'/g,"\""))
      var info =  JSON.parse(headerStr.toLowerCase().replace('(','[').replace(',),',']').replace('),',']').replace(/'/g,"\""));
      
      // Intepret the bytes according to the specified dtype
      console.log(info)
      var data;
      if (info.descr === "|u1") {
          data = new Uint8Array(buf, offsetBytes);
      } else if (info.descr === "|i1") {
          data = new Int8Array(buf, offsetBytes);
      } else if (info.descr === "<u2") {
          data = new Uint16Array(buf, offsetBytes);
      } else if (info.descr === "<i2") {
          data = new Int16Array(buf, offsetBytes);
      } else if (info.descr === "<u4") {
          data = new Uint32Array(buf, offsetBytes);
      } else if (info.descr === "<i4") {
          data = new Int32Array(buf, offsetBytes);
      } else if (info.descr === "<f4") {
          data = new Float32Array(buf, offsetBytes);
      } else if (info.descr === "<f8") {
          data = new Float64Array(buf, offsetBytes);
      } else {
          throw new Error('unknown numeric dtype')
      }

      return {
          shape: info.shape,
          fortran_order: info.fortran_order,
          data: data ,
          get : function(x,y){
              if (this.shape.length == 2)
                  return this.data[x*this.shape[0]+y]
              if (this.shape.length == 1)
                  return this.data[x]},
          dim : function(){
              return this.shape.length}
          
      };
}

define('hello', ["jupyter-js-widgets"], function(widgets) {

    var HelloView = widgets.DOMWidgetView.extend({

        render: function() {
            this.value_changed()
            
            this.model.on('change:value', this.value_changed, this);
        },

        value_changed: function() {
            if (this.model.get('byte_bigv')){
                var arr =  fromArrayBuffer(this.model.get('byte_bigv').buffer)
                console.log(arr.dim())
                if (arr.dim() == 1)
                    this.el.textContent =arr.get(1999)//
                
                else
                    this.el.textContent = arr.get(1,999)//
                
            }

            else{
                this.el.textContent = this.model.get('array_bigv')[0,999]
            }
            //console.log(this.model.get('byte_bigv').buffer)
        },
    });

    return {
        HelloView : HelloView
    };
});

<IPython.core.display.Javascript object>

In [269]:
import io

def doit(shape=(1000,1000),byte=True):
    Val = np.zeros(shape) 

    if type(shape) != int and len(shape) == 2:
        Val[0] = np.arange(shape[0])
        Val[1] = shape[0] + np.arange(shape[0])
    else:
        Val[0:2000] = np.arange(2000)
    #print(Val[1,0])
    if byte:
        iobyte = io.BytesIO()
        np.save(iobyte,Val)
        #print(Val.reshape(shape[0]*shape[1])[1000])
        A = HelloWidget(byte_bigv=iobyte.getvalue())
    else:
        A = HelloWidget(array_bigv=Val)
        

    return A

%timeit -n 10 A = doit((1000000))
%timeit -n 10 A = doit((1000,1000))

%timeit -n 1 A = doit((1000,1000),byte=False)


A = doit((1000000))
B = doit((1000,1000))

display(A)
display(B)

10 loops, best of 3: 12 ms per loop
The slowest run took 23.10 times longer than the fastest. This could mean that an intermediate result is being cached.
10 loops, best of 3: 14.4 ms per loop
1 loop, best of 3: 8.04 s per loop
